In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from architecture.classifier import DeepBinaryClassifier
from architecture.nodes.quine import make_quine_node

In [13]:
small_dataset_df = pd.read_csv("./data/10_bit_artificial/107.csv")
small_X_df = small_dataset_df.drop(columns="class").to_numpy(bool)
small_y_df = small_dataset_df["class"].to_numpy(bool)

small_X_train, small_X_test, small_y_train, small_y_test\
    = train_test_split(small_X_df, small_y_df, test_size=0.2, random_state=42, stratify=small_y_df)

large_dataset_df = pd.read_csv("./data/100_bit_artificial/1a.csv")
large_X_df = large_dataset_df.drop(columns="class").to_numpy(bool)
large_y_df = large_dataset_df["class"].to_numpy(bool)

large_X_train, large_X_test, large_y_train, large_y_test\
    = train_test_split(large_X_df, large_y_df, test_size=0.2, random_state=42, stratify=large_y_df)

print("Small dataset shapes:")
print(f"X_train: {small_X_train.shape}, X_test: {small_X_test.shape}")
print(f"y_train: {small_y_train.shape}, y_test: {small_y_test.shape}")

print("\nLarge dataset shapes:")
print(f"X_train: {large_X_train.shape}, X_test: {large_X_test.shape}")
print(f"y_train: {large_y_train.shape}, y_test: {large_y_test.shape}")

Small dataset shapes:
X_train: (819, 10), X_test: (205, 10)
y_train: (819,), y_test: (205,)

Large dataset shapes:
X_train: (8000, 100), X_test: (2000, 100)
y_train: (8000,), y_test: (2000,)


In [17]:
config = dict(
    layer_node_counts=[8]*3 + [1],
    layer_bit_counts=[8]*4,
    seed=41
)

X_train = small_X_train
y_train = small_y_train
X_test = small_X_test
y_test = small_y_test

net = DeepBinaryClassifier(**config, node_factory=make_quine_node, jobs=8)
%time net.fit(X_train, y_train)
pred_test = net.predict(X_test)
acc_test = accuracy_score(y_test, pred_test)
print(f"Accuracy on test set: {acc_test:.6f}")

CPU times: user 60 ms, sys: 128 ms, total: 188 ms
Wall time: 7.44 s
Accuracy on test set: 0.965854


In [18]:
node_truth_table, col_names = net.layers[1][1].get_truth_table()
node_truth_table_df = pd.DataFrame(node_truth_table, columns=col_names)
node_truth_table_df

,L1N0,L1N1,L1N2,L1N3,L1N4,L1N5,L1N6,L1N7,L2N1 (output)
0,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,False,True
2,True,True,True,True,True,True,False,True,True
3,True,True,True,True,True,True,False,False,True
4,True,True,True,True,True,False,True,True,True
...,...,...,...,...,...,...,...,...,...
251,False,False,False,False,False,True,False,False,False
252,False,False,False,False,False,False,True,True,False
253,False,False,False,False,False,False,True,False,False
254,False,False,False,False,False,False,False,True,False


In [19]:
# this node has an interesting expression
node = net.layers[1][1] # layer indexing seems fucked up too


print(node.get_expression())
node.reduce_expression()
print(node.get_expression())

node_truth_table, col_names = node.get_truth_table()
node_truth_table_df = pd.DataFrame(node_truth_table, columns=col_names)
node_truth_table_df

(L1N0 & L1N5) | (L1N5 & L1N7) | (L1N0 & L1N4 & L1N7) | (L1N2 & L1N4 & L1N5) | (L1N0 & L1N3 & ~L1N1) | (L1N1 & L1N7 & ~L1N3) | (L1N2 & L1N4 & ~L1N6) | (L1N1 & L1N7 & ~L1N0 & ~L1N2 & ~L1N6)
(L1N0 & L1N5) | (L1N5 & L1N7) | (L1N0 & L1N4 & L1N7) | (L1N2 & L1N4 & L1N5) | (L1N0 & L1N3 & ~L1N1) | (L1N1 & L1N7 & ~L1N3) | (L1N2 & L1N4 & ~L1N6) | (L1N1 & L1N7 & ~L1N0 & ~L1N2 & ~L1N6)


,L1N0,L1N1,L1N2,L1N3,L1N4,L1N5,L1N6,L1N7,L2N1 (output)
0,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,False,True
2,True,True,True,True,True,True,False,True,True
3,True,True,True,True,True,True,False,False,True
4,True,True,True,True,True,False,True,True,True
...,...,...,...,...,...,...,...,...,...
251,False,False,False,False,False,True,False,False,False
252,False,False,False,False,False,False,True,True,False
253,False,False,False,False,False,False,True,False,False
254,False,False,False,False,False,False,False,True,False
